In [1]:
import looker_sdk
from looker_sdk.sdk.api40.models import User
from typing import List
import pandas as pd
import datetime

In [ ]:
def explode_dicts_to_cols(df, col_to_explode):
    '''
    Creates one pandas DataFrame column per key in a dictionary if the column 
    has any dictionary values. Updates the column name as `{original_column_name}_{key}`
    '''
    assert any(type(df[col_to_explode].iloc[i]) == dict for i in range(len(df[col_to_explode])))
    return pd.concat([df.drop([col_to_explode], axis=1), df[col_to_explode].apply(pd.Series).add_prefix(f'{col_to_explode}_')], axis=1)

In [3]:
sdk = looker_sdk.init40()

In [9]:
all_users = sdk.all_users()

In [10]:
# Create a skeleton dict to fill with data
user_shell = User().__dict__

In [11]:
# fill the shell with empty lists to prepare for appending data to each list
user_shell_lists = {k: [] for k,v in user_shell.items()}

In [12]:
# get all data into the lists for each key
for user in all_users:
    for key in user_shell_lists.keys():
        try:
            # try to add the full object in cases where the value of an attribute is another object
            user_shell_lists[key].append(user[key].__dict__)
        except:
            user_shell_lists[key].append(user[key])

In [13]:
user_df = pd.DataFrame(user_shell_lists)

In [15]:
# Need to explode all dictionaries into columns
for col in user_df.columns:
    # If the type of any value in a cell is a dict, explode_dicts_to_cols
    if any(type(user_df[col].iloc[i]) == dict for i in range(len(user_df[col]))):
        user_df = explode_dicts_to_cols(user_df, col)

C:\Users\EricHeidbreder\AppData\Local\Temp\ipykernel_13280\3696290204.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.concat([df.drop([col_to_explode], axis=1), df[col_to_explode].apply(pd.Series).add_prefix(f'{col_to_explode}_')], axis=1)
C:\Users\EricHeidbreder\AppData\Local\Temp\ipykernel_13280\3696290204.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.concat([df.drop([col_to_explode], axis=1), df[col_to_explode].apply(pd.Series).add_prefix(f'{col_to_explode}_')], axis=1)
C:\Users\EricHeidbreder\AppData\Local\Temp\ipykernel_13280\3696290204.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.concat([d

In [18]:
# Set id as an int so we can sort it properly
user_df['id'] = user_df['id'].astype(int)
# Set index as id and sort it
user_df = user_df.set_index('id').sort_index()

In [19]:
# Get the columns that show the last logged in date
logged_in_columns = user_df.columns[user_df.columns.str.contains('logged_in_at')]

In [20]:
# Convert logged in cols to datetime so we can sort by them
user_df[logged_in_columns] = user_df[logged_in_columns].apply(lambda x: pd.to_datetime(x, utc=False))

In [23]:
# Combine the multiple login columns into one by taking the max
user_df['last_login_date'] = user_df[logged_in_columns].apply(lambda x: x.max(), axis=1)
# Combine the multiple looker employee columns by taking the max
user_df['is_looker_employee'] = user_df[['presumed_looker_employee', 'verified_looker_employee']].apply(lambda x: x.max(), axis=1)

In [24]:
number_of_days_considered_inactive = 365 # This is what we decided on for one client, but you can make this whatever you'd like

# Creating filters for final table
user_is_not_looker_employee = ~user_df['is_looker_employee']
user_is_not_disabled_filter = ~user_df['is_disabled']
user_has_not_logged_in_to_looker_this_year = user_df['last_login_date'].dt.date < (datetime.datetime.utcnow() - datetime.timedelta(days=number_of_days_considered_inactive)).date()

In [25]:
# Creating the final list
inactive_users = user_df[
    user_is_not_looker_employee &\
    user_is_not_disabled_filter &\
    user_has_not_logged_in_to_looker_this_year\
][['display_name', 'credentials_email_email', 'credentials_looker_openid_email', 'credentials_saml_email', 'last_login_date']]

In [31]:
# needs a folder named output, this will try to make it, and if it exists, we will skip the step
try:
    os.makedirs('./output/')
except:
    pass
inactive_users.to_csv(f'./output/users_who_havent_logged_in_in_{number_of_days_considered_inactive}_days.csv', index_label='user_id')